In [3]:
#  统计关系字典与实体字典的所有实体歧义类型
import os
import json
from nltk import pos_tag
from collections import Counter

def writeJsonFile(dictLineJson, write_json_file_path):
    with open(write_json_file_path, "a",encoding="utf-8") as fp:
        json.dump(dictLineJson, fp)  # write file
        fp.write("\n")

def readJsonFile(read_json_file_path):
    with open(read_json_file_path, "r",encoding="utf-8") as fp:
        json_dict = json.load(fp)
    return json_dict

def readPickleData(read_pkl_path):
    with open(read_pkl_path,"rb") as fp:
        pkl_data = pickle.load(fp)
    return pkl_data

def readLineFile(read_file_path):
    with open(read_file_path,"r",encoding="utf-8") as fp:
        lines = fp.readlines()
    return lines

def writeFile(str , write_file_path):
    with open(write_file_path,"a",encoding="utf-8") as fp:
        fp.write(str + "\n")

# statistics between  Relation Dict and Name_Category Dict different entity ratio!!!
def statERandEDictCategoryDifferentRatio(black_entity_path, entity_relation_dict_dir, name_category_entity_path,
                                         write_entity_relation_dict_dir, write_unexist_entity_path):
    name_category_dict = readJsonFile(name_category_entity_path)
    entity_relation_dict_list = os.listdir(entity_relation_dict_dir)
    statistics_different_entity_category = 0
#    different_entity_category_dict = {}
    unexist_entity_set = set()
#    category_statis_dict = {"compound": {}, "disease": {}, "drugs": {}, "gene": {}, "organism": {}, "protein": {}}
    black_entity_set = set()
    black_dict_lines = readLineFile(black_entity_path)
    for black_entity in black_dict_lines:
        black_entity_set.add(black_entity.strip())

    for entity_relation_file_name in entity_relation_dict_list:
        if entity_relation_file_name == ".DS_Store" or entity_relation_file_name ==".ipynb_checkpoints":
            continue
        print("compare {} file".format(entity_relation_file_name))
        entity_relation_dict_path = entity_relation_dict_dir + entity_relation_file_name
        entity_relation_dict_lines = readLineFile(entity_relation_dict_path)
        c1 = entity_relation_file_name.split("_")[0]
        c2 = entity_relation_file_name.split("_")[1]
        for line in entity_relation_dict_lines:
            entity_category_dict = {} # temp save different category of entity, which relation dict and name_category dict!
            #print("line =",line)
            line = json.loads(line)
            e1 = line["e1"]
            e2 = line["e2"]
            source = line["sources"]  # list ["-2","-4"]
            if e1 == "" or e2 == "":
                continue
            e1_flag = e1 in name_category_dict
            e2_flag = e2 in name_category_dict
            if e1_flag and c1 != name_category_dict[e1]:
                statistics_different_entity_category += 1
                # category_statis_dict[c1][e1] = name_category_dict[e1]
                entity_category_dict[e1] = c1 + "_" + name_category_dict[e1]
                entity_category_dict["source"] = source
                # different_entity_category_dict[e1] = c1 + "_"+name_category_dict[e1]
            if not e1_flag and e1 not in black_entity_set:
                if not (pos_tag([e1])[0][-1] in ["NN", "NNS", "NNP", "NNPS"] or (
                        pos_tag([e1])[0][-1] in ["JJ"] and (',' in e1 or '-' in e1))):
                    unexist_entity_set.add((e1, c1, str(source)))
            if e2_flag and c2 != name_category_dict[e2]:
                statistics_different_entity_category += 1
                # category_statis_dict[c2][e2] = name_category_dict[e2]
                # different_entity_category_dict[e1] = c2 + "_" + name_category_dict[e2]
                entity_category_dict[e2] = c2 + "_" + name_category_dict[e2]
                entity_category_dict["source"] = source
            if not e2_flag and e2 not in black_entity_set:
                if not (pos_tag([e2])[0][-1] in ["NN", "NNS", "NNP", "NNPS"] or (
                        pos_tag([e2])[0][-1] in ["JJ"] and (',' in e2 or '-' in e2))):
                    unexist_entity_set.add((e2, c2, str(source)))
            if entity_category_dict != {}:
                writeJsonFile(entity_category_dict, write_entity_relation_dict_dir + entity_relation_file_name)
        print("finish {} file".format(entity_relation_file_name))

    print("实体关系不在实体字典与黑名单中的实体个数", len(unexist_entity_set))

    writeJsonFile(list(unexist_entity_set), write_unexist_entity_path)#todo: 存储json格式的不能存储set数据，而set数据中不能存储list数据！！！


def statisDifferCategory(category_statis_dict_path):
    category_statis_dict = readJsonFile(category_statis_dict_path)
    counter = Counter()
    for key, value in category_statis_dict.items():
        value_value = value.values()
        category_stat = Counter(value_value)
        print("实体关系中属于{}而实体字典中不属于它的实体个数{} ".format(key, len(value)), category_stat)

def trimGiveLiJunData(read_trim_data_path, write_trim_data_path):
    trim_dict = readJsonFile(read_trim_data_path)
    # trim_dict = dict(trim_dict)
    trim_value_dict = trim_dict["protein"]
    line_flag = 0
    for key, value in trim_value_dict.items():
        if line_flag == 0:
            title = "entity" + "  " + "relation_dict_category" + "    " + "entity_dict_category"
            writeFile(title, write_trim_data_path)
        data_line = key + "   " + "protein" + "   " + value
        writeFile(data_line, write_trim_data_path)
        line_flag += 1

dict_num = {'-1': 'ttd', '-2': 'kegg', '-3': 'dgidb', '-4': 'drugbank', '-5': 'brenda', '-6': 'uniprot', '-7': 'chembl', '-8': 'drugcentral', '-9': 'malacards', '-10': 'ctd', '-11': 'biogrid', '-12': 'ebi', '-13': 'omim', '-14': 'disgenet', '-15': 'gene_organizer'}

def combineCategoryDifferentDict(read_combine_dict_dir,write_combine_finish_dir,write_combine_chihui_dir):
    combine_file_name_list = os.listdir(read_combine_dict_dir)
    for combine_file_name in combine_file_name_list:
        if combine_file_name == ".ipynb_checkpoints":
            continue
        combine_file_path = read_combine_dict_dir + combine_file_name
        lines = readLineFile(combine_file_path)
        combine_different_category_dict = {}
        c1 = combine_file_name[:-5].split("_")[0]
        c2 = combine_file_name[:-5].split("_")[1]
        record_summary_dict_1 ={c1:""}
        record_summary_dict_2 = {c2: ""}
        record_summary_dict_3 = {c1 + "_" + c2 : ""}
        count_1 = Counter()
        count_2 = Counter()
        count_3 = Counter()
        for line in lines:
            line = json.loads(line)
            c3_1 = ""
            c3_2 = ""
            for key, value in line.items():
                if key != "source":
                    combine_different_category_dict[key] = value
                    value_c1 = value.split("_")[0]
                    value_c1_1 = value.split("_")[1]
                    #一行中e1实体不相同的
                    if value_c1 == c1:
                        source_list = sorted(line["source"])
                        for source_id in source_list:
                            count_1[value_c1_1 +"("+ dict_num[source_id]+")"] += 1
                    #一行中e2实体不相同的
                    if value_c1 == c2:
                        source_list = sorted(line["source"])
                        for source_id in source_list:
                            count_2[value_c1_1 +"("+ dict_num[source_id]+")"] += 1
                    #一行两个实体的类别都不相同都
                    if len(line) == 3:

                        if value_c1 == c1:
                            c3_1 = value_c1_1
                            continue
                        if value_c1 == c2:
                            c3_2 = value_c1_1
                        source_list = sorted(line["source"])
                        for source_id in source_list:
                            count_3[c3_1+"_"+c3_2 + "(" + dict_num[source_id]+")"] += 1

        record_summary_dict_1[c1]=count_1
        record_summary_dict_2[c2] = count_2
        record_summary_dict_3[c1 + "_" + c2] = count_3
        write_combine_finish_file_path = write_combine_finish_dir + combine_file_name
        writeJsonFile(record_summary_dict_1,write_combine_finish_file_path)
        writeJsonFile(record_summary_dict_2, write_combine_finish_file_path)
        writeJsonFile(record_summary_dict_3,write_combine_finish_file_path)


        #给迟慧将文件中不同的类别总结在一个字典中
        write_combine_chihui_file_path = os.path.join(write_combine_chihui_dir , combine_file_name)
        writeJsonFile(combine_different_category_dict, write_combine_chihui_file_path)

if __name__ == "__main__":
    #todo: statistics the information, which entity category of entity and name_category dict different!!!
    black_entity_path = "./Data/black_dict_v3.0.txt"
#    entity_relation_dict_dir = "./Data/NewEntityRelationDict_v2.1/"
#    entity_relation_dict_dir = "./relation_entity_dict_align_v3.0/"
    entity_relation_dict_dir = "./all_relation_dict_file_v2.0/quchong_relation_dict_v2.1/"
    name_category_entity_path = "./Data/medical_entity_Name_Category_v4.4.json"
    write_entity_relation_dict_dir = "./Data/EntityRelationDictInfo_v3.3/" # save information of  differnet entity category!
    write_unexist_entity_path = "./Data/unexist_entity_v3.3.json"
    statERandEDictCategoryDifferentRatio(black_entity_path, entity_relation_dict_dir, name_category_entity_path,
                                         write_entity_relation_dict_dir, write_unexist_entity_path)

    #todo: statistics in detail!!!
    # read_trim_data_path = "./Data/category_statis_dcit.json"
    # write_trim_data_path  = "./SatisticsData/不正常的实体类别统计.txt"
    # trimGiveLiJunData(read_trim_data_path, write_trim_data_path)

    #todo: combine different entity category for chihui
    read_combine_dict_dir = "./Data/EntityRelationDictInfo_v3.3/"
    write_combine_finish_dir = "./Data/SummaryLiJunDifferentCategoryDict_v3.3/"
    write_combine_chihui_dir = "./Data/SummaryChihuiDifferentCategoryDict_v3.3/"
    combineCategoryDifferentDict(read_combine_dict_dir, write_combine_finish_dir, write_combine_chihui_dir)
    

compare compound_protein_treat.dict file
finish compound_protein_treat.dict file
compare gene_organism_belong.dict file
finish gene_organism_belong.dict file
compare equipment_disease_diagnose|treat.dict file
finish equipment_disease_diagnose|treat.dict file
compare drugs_anatomy_affect.dict file
finish drugs_anatomy_affect.dict file
compare compound_disease_cause.dict file
finish compound_disease_cause.dict file
compare gene_disease_cause.dict file
finish gene_disease_cause.dict file
compare gene_protein_express.dict file
finish gene_protein_express.dict file
compare drugs_organism_apply.dict file
finish drugs_organism_apply.dict file
compare disease_anatomy_occur.dict file
finish disease_anatomy_occur.dict file
compare compound_drugs_contain.dict file
finish compound_drugs_contain.dict file
compare protein_anatomy_belong.dict file
finish protein_anatomy_belong.dict file
compare gene_gene_interact.dict file
finish gene_gene_interact.dict file
compare gene_anatomy_locate.dict file
fini